In [1]:
from tools import OUT_PATH, open_meta_df
import pandas as pd
import numpy as np
from skopt import BayesSearchCV
from sklearn.neighbors import KNeighborsClassifier
from tqdm.auto import tqdm
from skopt.space import Integer, Categorical, Integer
from pprint import pprint
from warnings import filterwarnings
from joblib import dump
filterwarnings("ignore")

In [2]:
meta_df = open_meta_df()
videos_centroids = np.load(OUT_PATH / 'videos_centroids.npy')
fold_path = OUT_PATH / 'Folds'
train_videos = [np.load(x) for x in sorted(fold_path.glob('*/train.npy'))]
test_videos = [np.load(x) for x in sorted(fold_path.glob('*/test.npy'))]
len(train_videos)

5

In [3]:
reshaped_videos_centroids = videos_centroids.reshape((videos_centroids.shape[0], -1))
reshaped_videos_centroids.shape

(1058, 990)

In [4]:
def get_sample(index):
    return reshaped_videos_centroids[index], meta_df.loc[index, 'pose_id']

In [5]:
scores = [None] * len(train_videos)

for i, (train, test) in enumerate(tqdm(list(zip(train_videos, test_videos)))):
    opt = BayesSearchCV(
        KNeighborsClassifier(n_jobs=-1),
        {
            'n_neighbors': Integer(1, 100),
            'weights': Categorical(['uniform', 'distance']),
        },
        n_iter=32,
        random_state=42,
        cv=3,
        n_jobs=-1,
        verbose=0,
    )
    opt.fit(*get_sample(train))
    result = {
        'best_score': opt.best_score_,
        'best_params': opt.best_params_,
        'best_index': opt.best_index_,
        'cv_results': opt.cv_results_,
        'test_score': opt.score(*get_sample(test)),
    }
    scores[i] = result
pprint(scores)

  0%|          | 0/5 [00:00<?, ?it/s]

[{'best_index': 9,
  'best_params': OrderedDict([('n_neighbors', 1), ('weights', 'uniform')]),
  'best_score': 0.24231678486997635,
  'cv_results': {'mean_fit_time': array([0.00255116, 0.0027705 , 0.00266592, 0.00257754, 0.00267744,
       0.00302585, 0.00314411, 0.00265638, 0.00273832, 0.00321356,
       0.00246914, 0.00223509, 0.00244784, 0.00228119, 0.00230622,
       0.00253097, 0.00233936, 0.0023071 , 0.00219893, 0.00226895,
       0.00229979, 0.00224694, 0.00234667, 0.00217533, 0.00223859,
       0.00233301, 0.00226426, 0.00237815, 0.00219154, 0.00222802,
       0.00224415, 0.00232506]),
                 'mean_score_time': array([0.05760304, 0.05272643, 0.04799223, 0.04461106, 0.04783805,
       0.0184044 , 0.01552947, 0.0130167 , 0.01462865, 0.01237782,
       0.01380952, 0.01353598, 0.00835975, 0.00962607, 0.00845313,
       0.0088466 , 0.0138979 , 0.00994205, 0.01216698, 0.01412813,
       0.01129444, 0.00822449, 0.01330328, 0.01250203, 0.01102742,
       0.01897478, 0.0082556

In [6]:
dump(scores, OUT_PATH / 'scores/knn_scores.joblib')

['Outs/scores/knn_scores.joblib']